# Prod2vec runbook

## Loading libraries and setting parameters

In [ ]:
import sys

sys.path.append("../../..")

from src.common import config
from src.models.prod2vec import Prod2Vec
from src.evaluation.evaluator import Evaluator

%load_ext autoreload

In [2]:
paths = config.Paths(
    dataset_name="jobs_published-part",
    target_users_name="subset_validation",
    model_name="prod2vec",
)

In [3]:
VECTOR_SIZE = 48
EPOCHS = 6

N_RECOMMENDATIONS = 10

## Model training

In [4]:
%autoreload
model = Prod2Vec(vector_size=VECTOR_SIZE, epochs=EPOCHS)

model.load_interactions(paths.train)

model.preprocess()

model.fit()

## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS, show_progress=True
)

model.save_recommendations(recommendations, paths.recommendations)

17857it [00:05, 3214.38it/s]


## Evaluation

In [6]:
%autoreload
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=["prod2vec"],
)

evaluator.prepare()

evaluator.evaluate_models()

17857it [00:01, 16214.11it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
prod2vec,0.005634,0.034209,0.009208,0.006646,0.018066,0.010855,0.016616,0.516431,0.0518,0.427143,0.685175,7.697137,0.765819,0.001456,0.001456
